In [1]:
import pandas as pd
from datetime import date
from datetime import datetime, timedelta
from splinter import Browser
from selenium import webdriver
from time import sleep
import random
import math
! pip install selenium
# Definir fecha de referencia para buscar empleos posteriores a esta fecha Ej: -4 trae los resltados de los últimos 4 dias
today = (date.today()+ timedelta(days=-4)).strftime('%Y %m %d')

In [2]:
Fecha=[]
Cargo=[]
Salario=[]
Empresa=[]
Link=[]
Guia=[]

#Definicion del driver de chrome
driver = webdriver.Chrome(r'C:\Users\Samuel Cuartas\Tesis/chromedriver.exe')# Nota hay que descargar el driver de chrome e instalarlo

urls = [
    'https://www.elempleo.com/co/ofertas-empleo/medellin?&trabajo=Analitica',
    'https://www.elempleo.com/co/ofertas-empleo/Medellin?&trabajo=Business%20Intelligence',
    'https://www.elempleo.com/co/ofertas-empleo/Medellin?&trabajo=Financiero',
    'https://www.elempleo.com/co/ofertas-empleo/medellin?trabajo=Project%20Manager',
    'https://www.elempleo.com/co/ofertas-empleo/medellin?trabajo=product%20owner',
    'https://www.elempleo.com/co/ofertas-empleo/Medellin?&trabajo=Inteligencia%20de%20Negocios'    
]

for url in urls:
    driver.get(url)
    cant_result= int(driver.find_element_by_xpath('//strong[@class="js-total-results"]').text)# Numero total de resultados de la busqueda
    cant_rondas=int((math.ceil(cant_result / 100)) * 100/100) #Calculo del número de páginas de la busqueda
    try:
        driver.find_element_by_xpath('//select[@class="form-control js-results-by-page"]/option[text()=100]').click()
    except Exception:
        pass
    
    sleep(random.uniform(8,12))
    for i in range (cant_rondas):
        try:
            empleos = driver.find_elements_by_xpath('//div[@class="result-item"]')# Listado de los empleos de la página
            for empleo in empleos:
                fecha= empleo.find_element_by_xpath('.//span[@class="info-publish-date pull-right js-offer-date"]').text
                Fecha.append(fecha) # Fecha
                #print(fecha)
                cargo= empleo.find_element_by_xpath('.//h2/a').text
                Cargo.append(cargo) # Cargo
                #print(cargo)
                salario= empleo.find_element_by_xpath('.//span[@class="text-primary info-salary js-offer-salary"]').text
                Salario.append(salario) # Salario
                #print (salario)
                empresa = empleo.find_element_by_xpath('.//span[@class="info-company-name js-offer-company"]').text
                Empresa.append(empresa) #Empresa
                #print (empresa)
                link = empleo.find_element_by_xpath('.//h2/a').get_attribute('href')
                Link.append(link) #Link de la url oferta
                #print (link)
                Guia.append(url) # Link de la url de busqueda
            driver.execute_script("arguments[0].click();", driver.find_element_by_xpath('//a[@class="js-btn-next"]'))
            sleep(random.uniform(8.0,12.0))
        except:
            break

In [3]:
# Creacion de Dataframe
df = pd.DataFrame(list(zip(Fecha, Cargo, Salario, Empresa, Link, Guia)),
               columns =['Fecha', 'Cargo', 'Salario','Empresa','Link','Guia'])
df['Fecha']=df['Fecha'].str.join('').str.replace(r"\r\n", "")
df['Fecha']=df['Fecha'].str.join('').str.replace("'", "")
df['Fecha']=df['Fecha'].str.join('').str.replace("Publicado", "")
df['Fecha']=df['Fecha'].str.strip()
df['Fecha']=df.Fecha.replace({'Ene':'01','Feb':'02','Mar':'03','Abr':'04','May':'05','Jun':'06',
                 'Jul':'07','Ago':'08','Sep':'09','Oct':'10','Nov':'11','Dic':'12'}, regex=True)
df['Fecha']=pd.to_datetime(df['Fecha'], format='%d %m %Y').dt.strftime('%Y %m %d')
df = df.sort_values(by="Fecha", ignore_index=True,ascending=False)
df=df.drop_duplicates(subset=['Fecha', 'Cargo', 'Salario'])
df=df[(df.Salario != 'Menos de $1 millón' ) &(df.Salario != '$1 a $1,5 millones' ) & (df.Salario != '$1,5 a $2 millones') 
      & (df.Salario != '$2 a $2,5 millones')& (df.Salario != '$2,5 a $3 millones')]
df = df.loc[(df['Fecha'] >= today)]

In [4]:
#Exportación a excel
df.to_excel("empleos.xlsx")

In [5]:
# Envío de resultados a correo
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
mail_content = '''Hello,
This is a test mail.
In this mail we are sending some attachments.
The mail is sent using Python SMTP library.
Thank You
'''
#The mail addresses and password
sender_address = ''
sender_pass = ''
receiver_address = ''
#Setup the MIME
message = MIMEMultipart()
message['From'] = sender_address
message['To'] = receiver_address
message['Subject'] = 'A test mail sent by Python. It has an attachment.'
#The subject line
#The body and the attachments for the mail
message.attach(MIMEText(mail_content, 'plain'))
attach_file_name = 'empleos.xlsx'
attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
payload = MIMEBase('application', 'vnd.ms-excel')
payload.set_payload((attach_file).read())
encoders.encode_base64(payload) #encode the attachment
#add payload header with filename
payload.add_header('Content-Decomposition', 'attachment', filename=attach_file_name)
message.attach(payload)
#Create SMTP session for sending the mail
session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
session.starttls() #enable security
session.login(sender_address, sender_pass) #login with mail_id and password
text = message.as_string()
session.sendmail(sender_address, receiver_address, text)
session.quit()
print('Mail Sent')

Mail Sent
